Copyright © 2017-2023 ABBYY

In [1]:
#@title
# 
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Import from ONNX

[Download the tutorial as a Jupyter notebook](https://github.com/neoml-lib/neoml/blob/master/NeoML/docs/en/Python/tutorials/ONNX.ipynb)

In this tutorial, we'll import an ONNX model to NeoML and validate the its' return. We'll also test additional more advanced settings of ONNX import.

The tutorial includes the following steps:

* [Download the model](#Download-the-model)
* [Import with deffault settings](#Import-with-default-setttings)
* [Import with different layouts](#Import-with-different-layouts)

## Download the model

For this tutorial we will use MobilNetV2 model from ONNX model zoo

In [2]:
import requests

url = 'https://github.com/onnx/models/raw/main/vision/classification/mobilenet/model/mobilenetv2-12.onnx'
file_name = 'mobilenetv2.onnx'

with requests.get(url, stream=True) as url_stream:
    url_stream.raise_for_status()
    with open(file_name, 'wb') as file_out:
        for chunk in url_stream.iter_content(chunk_size=8192):
            file_out.write(chunk)

## Import with default settings

ONNX uses tensors with variable number of dimensions. NeoML uses blobs with fixed number of dimensions (7). The default way ONNX input and output tensors are emulated in NeoML is by using first dimensions of NeoML blobs.

This model has 1 ONNX input named `input` and 1 ONNX output named `output`.

The `input` expects `batch_size x 3 x 224 x 224` ONNX tensor which means `batch_size x 3 x 224 x 224 x 1 x 1 x 1` NeoML blob.

The `output` will be `batch_size x 1000` ONNX tensor which means `batch_size x 1000 x 1 x 1 x 1 x 1 x 1` NeoML blob.

In [3]:
# import the model with default settings
import neoml

math_engine = neoml.MathEngine.default_math_engine()
default_model, model_info = neoml.Onnx.load_from_file(file_name, math_engine)

In [4]:
# generate some input data
import numpy as np
input_data = np.linspace(0.0, 1.0, num=3 * 224 * 224, dtype=np.float32)
input_data.resize(1, 3, 224, 224)
input_blob = neoml.Blob.asblob(math_engine, input_data, [1, 3, 224, 224, 1, 1, 1])

output_blob = default_model.run({model_info['inputs'][0]:input_blob})[model_info['outputs'][0]]
print('output shape is ', output_blob.shape)
default_output = output_blob.asarray(copy=True)

output shape is  (1, 1000, 1, 1, 1, 1, 1)


## Import with different layouts

There is another difference between NeoML and ONNX. ONNX works with channel-first (NCHW) image format. NeoML works with channel-last (NHWC).

NeoOnnx will import the model in a way that input and output blobs will be having the same dimension order as ONNX inputs and outputs. But this will cause inputs to be transposed inside the network.

In cases when your pipeline allows to feed or get images in channel-last format you can import ONNX model with assumption that images will be in channel-last format and will result in a more optimal NeoML model.

In order to do this we need to use **layouts**. Layout of an ONNX tensor is an array of NeoML blob dimensions. The length of layout is equal to the number of dimensions of ONNX dimensions. `layouts[i]` sets in which blob dimension ONNX tensor's i'th dimension must be put.

Default layout for 4-dimensional ONNX tensor is 'first 4 dims of blob' which is equivalent to `layout = ['batch_length', 'batch_width', 'list_size', 'height']`.

But `batch_size x 3 x 224 x 224` ONNX tensor is actually a batch of RGB images. So when first ONNX tensor's dim is a batch, second is `channels`, third is `height` and the last is `width` NeoML won't be doing any additional transpositions and transformations.

In [5]:
to_channel_last = ['batch_width', 'channels', 'height', 'width']

opt_model, model_info = neoml.Onnx.load_from_file(file_name, math_engine, input_layouts={'input' : to_channel_last})
# with new layout expected blob is 1 x batch_size x 1 x 224 x 224 x 1 x 3
# and we need to transpose data from numpy accordingly
input_blob = neoml.Blob.asblob(math_engine, input_data.transpose((0, 2, 3, 1)), [1, 1, 1, 224, 224, 1, 3])

output_blob = opt_model.run({model_info['inputs'][0] : input_blob})[model_info['outputs'][0]]
opt_output = output_blob.asarray(copy=True)

print('max diff between opt and default outputs: ', np.max(opt_output - default_output))

max diff between opt and default outputs:  0.0


As you can see, the outputs are equal. Now, let's check how many transforms and tranpositions nets have

In [6]:
def print_onnx_helper_stat(net_name, dnn):
    n_transposes = 0
    n_transforms = 0
    for _, layer in dnn.layers.items():
        if layer.class_name == 'NeoMLDnnOnnxTransposeHelper':
            n_transposes += 1
        elif layer.class_name == 'NeoMLDnnOnnxTransformHelper':
            n_transforms += 1
    print(net_name)
    print('\ttranposes: ', n_transposes)
    print('\ttranforms: ', n_transforms)

print_onnx_helper_stat('default', default_model)
print_onnx_helper_stat('optimized', opt_model)

default
	tranposes:  2
	tranforms:  2
optimized
	tranposes:  1
	tranforms:  1


Providing input images as channel-last resulted in less conversions and transpositions!